In [1]:
import pandas as pd
from datetime import datetime

**Primero de todo hacer una copia de la tabla de la url separada en un CSV para poder trabajar con él**

In [2]:
clean_dataframe = pd.read_csv("df_navegacion_urlseparated_noduplicates_conversiones_concatenated_withtime.csv")

## 1. Informe de ratio de conversión


In [14]:
def get_campañas_data():
    campañas_list = clean_dataframe['camp'].unique()

    columns_list = ["campaña", "numero_conversiones", "numero_clicks", "ratio_conversion"]
    df_campañas_data = pd.DataFrame(columns = columns_list)

    campaña_list = []
    numero_conversiones_list = []
    numero_clicks_list = []
    ratio_conversion_list = []

    for campaña in campañas_list:
        if 'camp' in str(campaña):
            df_campaña = clean_dataframe.loc[clean_dataframe['camp'] == campaña]
            
            campaña_list.append(campaña)

            numero_conversiones = len([df_campaña['conversiones'] == 1])
            numero_conversiones_list.append(numero_conversiones)

            df_campaña = clean_dataframe.loc[clean_dataframe['camp'] == campaña]
            numero_clicks = len(df_campaña)
            numero_clicks_list.append(numero_clicks)

            ratio_conversion = numero_conversiones/numero_clicks * 100
            ratio_conversion_list.append(ratio_conversion)

    df_campañas_data['campaña'] = campaña_list
    df_campañas_data['numero_conversiones'] = numero_conversiones_list
    df_campañas_data['numero_clicks'] = numero_clicks_list
    df_campañas_data['ratio_conversion'] = ratio_conversion_list
    
    df_campañas_data.to_csv('ratio_conversion_campañas_data.csv', index = False)
    
    return df_campañas_data

                    
df_campañas_data = get_campañas_data()
df_campañas_data

,campaña,numero_conversiones,numero_clicks,ratio_conversion
0,camp=732187328,1,2122,0.047125
1,camp=1646697023,1,106,0.943396
2,camp=1648648995,1,1081,0.092507
3,camp=1648174978,1,13,7.692308
4,camp=13352855428,1,92,1.086957
...,...,...,...,...
95,camp=1439144630,1,1,100.000000
96,camp=1439144639,1,1,100.000000
97,camp=13461666926,1,1,100.000000
98,camp=283493082,1,1,100.000000


In [15]:
def get_informe_ratio_conversion():
    # Cálculo del ratio de conversión
    df_only_conversiones = clean_dataframe.loc[clean_dataframe["conversiones"] == 1]
    numero_clientes_conversion = len(df_only_conversiones)
    numero_clientes_total = len(clean_dataframe)
    ratio_conversion = numero_clientes_conversion/numero_clientes_total
    
    # Guardado de los registros que contienen conversiones = 1 (es decir que han convertido)
    df_only_conversiones.to_csv("df_only_conversiones.csv")
    
    return ratio_conversion
                    
ratio_conversion = get_informe_ratio_conversion()
print("El ratio de conversión es: " + str(ratio_conversion))

El ratio de conversión es: 0.0025870989996550535


## 2. Media de tiempo que tarda el call center en ponerse en contacto con el usuario cuando rellena un formulario

#### 2.1. Recoger todos los usuarios que tienen FORM en lead_type

In [5]:
call_clean_dataframe = clean_dataframe.loc[clean_dataframe['lead_type']==' FORM']
call_clean_dataframe.to_csv('CALL_contact.csv', index = False)
#call_clean_dataframe.drop(call_clean_dataframe.index[13])

In [6]:
def get_media_tiempo(dataframe):
    diferencia_horas_list = []
    for element in dataframe.itertuples():

        # Tiempo de la navegación
        navegacion_day = element.day
        navegacion_hour = element.hour.strip()
        splitted_navegacion_hour = navegacion_hour.split(':')
        navegacion_horas = int(splitted_navegacion_hour[0])
        navegacion_minutos = int(splitted_navegacion_hour[1])

        # Tiempo de la navegación en minutos
        time_navegacion_minutos = navegacion_day*24*60 + navegacion_horas*60 + navegacion_minutos

        # Tiempo de la conversión
        conversion_day = int(element.day_conversion.strip())
        conversion_hour = element.hour_conversion.strip()
        splitted_conversion_hour = conversion_hour.split(':')
        conversion_horas = int(splitted_conversion_hour[0])
        conversion_minutos = int(splitted_conversion_hour[1])

         # Tiempo de la conversión en minutos
        time_conversion_minutos = conversion_day*24*60 + conversion_horas*60 + conversion_minutos
        
        # Cálculo del tiempo que el call center tarda en contestar
        time_contestar = time_conversion_minutos - time_navegacion_minutos
        
        diferencia_horas_list.append(time_contestar)

    suma_time = 0
    for time in diferencia_horas_list:
        suma_time = suma_time + int(time)

    media_time = suma_time / len(call_clean_dataframe)

    print("LA MEDIA DE TIEMPO QUE EL CALL CENTER TARDA EN CONTESTAR ES DE: " + str(media_time/60) + ' HORAS Y ' + str(media_time%60) + ' MINUTOS' )
    
call_clean_dataframe = clean_dataframe.loc[clean_dataframe['lead_type']==' FORM']
get_media_tiempo(call_clean_dataframe)

LA MEDIA DE TIEMPO QUE EL CALL CENTER TARDA EN CONTESTAR ES DE: 23.496296296296297 HORAS Y 29.77777777777783 MINUTOS


## 3. Identificar el número de clientes que se pueden localizar por medio de CALL

In [7]:
call_users_dataframe = clean_dataframe.loc[clean_dataframe['lead_type']==' CALL']   
call_users_dataframe

,day,month,year,hour,uuid,id_user,gclid,user_recurrent,url_landing,URL_base&landing,...,rec,extra,conversiones,id_lead,lead_type,result,hour_conversion,day_conversion,month_conversion,year_conversion
5,6,9,2021,13:44:21.471,54bfdfe1-69ca-4e22-bf78-90923d1e9bf9,59257b75-5389-49e4-b1aa-5997ecfdc7f9,Cj0KCQjw1dGJBhD4ARIsANb6OdkF0-DcGTMMXczOIXbbjK...,True,https://www.metropolis.com/es/cea/gclid=Cj0KCQ...,https://www.metropolis.com/es/cea/,...,rec=true,NaN,1,7959,CALL,No le interesa,13:45,9,9,2021
71,6,9,2021,11:56:55.657,e70ae10c-c91d-46f9-9c44-6a470c7995dc,8fb314e8-b0fe-4e17-b398-c0c5d2a988d0,Cj0KCQjw1dGJBhD4ARIsANb6Odmm0scxrJhSEv-sJOjunw...,False,https://www.metropolis.com/es/clin200/gclid=Cj...,https://www.metropolis.com/es/clin200/,...,rec=true,NaN,1,7914,CALL,Ilocalizable,11:56,8,9,2021
371,6,9,2021,7:0:16.755,8c89b4be-197f-4664-b3d1-1f48d9b290c7,1d792022-78e4-4af9-9ea5-741117389e66,Cj0KCQjw-NaJBhDsARIsAAja6dM0p7PIAgtxJQ0hegy6Z-...,True,https://www.metropolis.com/es/clin400/gclid=Cj...,https://www.metropolis.com/es/clin400/,...,rec=true,NaN,1,7851,CALL,Ilocalizable,11:37,7,9,2021
854,6,9,2021,17:35:39.374,39e46fb6-ecd7-473c-96ff-25b92bd24529,fff88164-ce92-486c-a4d8-41c7b9779f24,EAIaIQobChMIquKwkIDq8gIVRLrVCh3JEwTmEAAYASAAEg...,False,https://www.metropolis.com/es/gclid=EAIaIQobCh...,https://www.metropolis.com/es/,...,rec=false,NaN,1,7989,CALL,No le interesa,17:36,10,9,2021
1794,6,9,2021,11:56:25.928,c4748bc6-eeec-4c94-ac60-2c5960748869,2f616f73-6d38-4041-95e9-911b90fbaee2,Cj0KCQjw-NaJBhDsARIsAAja6dM05lyJ5F1LMB-EMr_V51...,False,https://www.metropolis.com/es/gclid=EAIaIQobCh...,https://www.metropolis.com/es/,...,rec=false,NaN,1,7843,CALL,Ilocalizable,17:58,7,9,2021
4246,6,9,2021,20:01:18.455,427545ba-91a4-4e5d-8a69-895fefe2c0ef,fbee18ca-6970-4015-8892-814e664b3dab,Cj0KCQjw-NaJBhDsARIsAAja6dM1v0WCTK_OfE2RBzWB7r...,True,https://www.metropolis.com/es/clin400/gclid=EA...,https://www.metropolis.com/es/clin400/,...,rec=false,NaN,1,7836,CALL,No le interesa,20:02,6,9,2021


In [8]:
result_list = call_users_dataframe['result'].tolist()
print(result_list[1])

 Ilocalizable


In [9]:
call_users_dataframe = clean_dataframe.loc[clean_dataframe['lead_type']==' CALL'] 
localizable_call_users_dataframe = call_users_dataframe.loc[call_users_dataframe['result']!=' Ilocalizable']
localizable_call_users_dataframe.to_csv('localizable_call_users.csv', index = False)
localizable_call_users_dataframe


,day,month,year,hour,uuid,id_user,gclid,user_recurrent,url_landing,URL_base&landing,...,rec,extra,conversiones,id_lead,lead_type,result,hour_conversion,day_conversion,month_conversion,year_conversion
5,6,9,2021,13:44:21.471,54bfdfe1-69ca-4e22-bf78-90923d1e9bf9,59257b75-5389-49e4-b1aa-5997ecfdc7f9,Cj0KCQjw1dGJBhD4ARIsANb6OdkF0-DcGTMMXczOIXbbjK...,True,https://www.metropolis.com/es/cea/gclid=Cj0KCQ...,https://www.metropolis.com/es/cea/,...,rec=true,NaN,1,7959,CALL,No le interesa,13:45,9,9,2021
854,6,9,2021,17:35:39.374,39e46fb6-ecd7-473c-96ff-25b92bd24529,fff88164-ce92-486c-a4d8-41c7b9779f24,EAIaIQobChMIquKwkIDq8gIVRLrVCh3JEwTmEAAYASAAEg...,False,https://www.metropolis.com/es/gclid=EAIaIQobCh...,https://www.metropolis.com/es/,...,rec=false,NaN,1,7989,CALL,No le interesa,17:36,10,9,2021
4246,6,9,2021,20:01:18.455,427545ba-91a4-4e5d-8a69-895fefe2c0ef,fbee18ca-6970-4015-8892-814e664b3dab,Cj0KCQjw-NaJBhDsARIsAAja6dM1v0WCTK_OfE2RBzWB7r...,True,https://www.metropolis.com/es/clin400/gclid=EA...,https://www.metropolis.com/es/clin400/,...,rec=false,NaN,1,7836,CALL,No le interesa,20:02,6,9,2021


## 4. Identificar la lista de sus productos (coches) teniendo en cuenta el ratio de conversión

In [13]:
def get_products(dataframe):
    URLbase_landing = dataframe["URL_base&landing"]
    pages_list = get_pages_list(URLbase_landing)
    products_list = get_products_list(pages_list)
    
    return products_list
products_list = get_products(clean_dataframe)
print(products_list)

['clin200', 'renting', 'descarga-catalogo', 'life', 'ixs-hibrido', 'clin400', 'dep30', 'tria', 'cea', 'cea-electrico', 'comercial', 'fm', 'clin200n', 'clin50', 'clin200-electrico', 'fm-hev', 'cea-hibrido', 'tria-hibrido', 'dep30F', 'k43', 'ixs-electrico', 'descargar-catalogo']


In [11]:
def get_pages_list(URLbase_landing):
    pages_list = []
    for element in URLbase_landing:
        if isinstance(element, str):
            splitted_element = element.split("/")
            if splitted_element[len(splitted_element) - 2] == 'es':
                pages_list.append("Home")
            else:
                pages_list.append(splitted_element[len(splitted_element) - 2])
                
    return pages_list

In [12]:
def get_products_list(pages_list):
    pages_set = list(set(pages_list))
    products = []
    for product in pages_set:
        if 'User' not in product and 'Home' not in product and 'home' not in product:
            products.append(product)
            
    return products